# The Vexatious Nature of Levered Returns

First of all, note that maintaining leverage over some given horizon requires you to dynamically rebalance to target.

For example, suppose SPXL, Direxion's Daily 3x-Levered Bull SPX ETF, has USD1 NAV (USD1 client assets + USD2 borrowed money - USD2 owed on the loan), and hence invests USD3 in the SP500. Then suppose today the SP500 return 1pct (assume zero interest rates), growing SPXL's NAV to USD1.03.

This means Direxion must go back to the bank and borrow an additional USD0.06, because now (assuming they didn't buy or sell any of the shares, meaning that conveniently their holdings naturally evolved to reflect the new composition of the SP500, which is a convenient feature of market-cap-weighted benchmarks and indeed a cornerstone of truly "passive" investment strategies) they are technically at 2.94x=USD3.03/USD1.03 leverage which is slightly less than their 3x stated target.

Second of all, take a look at the below. Levering up to 1.20x seems like a good tradeoff: If you consistently win, you overall win fully 1.473x as much; whereas if you consistently lose, you overall lose only 1.068x as much. But levering down to 0.80x seems like a bad tradeoff: If you consistently win, you overall win only 0.657x as much; whereas if you consistently lose, you overall lose fully 0.905x as much. Note that the "contrapositive" therefore also holds: Based on this logic, levering up from 0.80x to 1.00x seems like a good tradeoff. The upshot is that deleveraging as a risk-reduction tactic might dampen your wins more than expected, while simultaneously dampening your losses less than expected.

For a more optimistic example of a similar financial "convexity" phenomenon, cf Aaron Brown at https://www.bloomberg.com/opinion/articles/2020-05-26/twitter-spat-over-market-risks-is-a-teaching-moment.

In [1]:
ANN = 261  # financial calendar daycount (# of trading days per year)
def cmp_pnl(lev=1, ltv=0.10, ann=ANN, sign=+1, std=1, dur=ANN):
    """
    Compare `lev`x-levered vs 1x-levered pnl
    if we realize `sign*std`STD pnl
    (at `ltv` annualized long-term vol)
    for `dur`days in a row.
    """
    def get_pnl(lev):
        print ("\tleverage = {lev}x".format(lev=lev))
        daily_pnl = lev * sign * std * ltv / ann**0.5
        px = (1 + daily_pnl)**dur
        pnl = px - 1
        print("\t\tfinal price index = ${px}".format(px=round(px, 2)))
        print("\t\tfinal pnl = {pnl}%".format(pnl=round(100*pnl, 1)))
        return pnl
    unlev_pnl = get_pnl(lev=1)
    lev_pnl = get_pnl(lev=lev)
    print("\t>>>> ratio = {} <<<<".format(round(lev_pnl / unlev_pnl, 3)))

for lev in 1.20, 0.80:
    for sign in +1, -1:
        print("SIGN = {sign}".format(sign=sign))
        cmp_pnl(lev=lev, sign=sign)
    print()

SIGN = 1
	leverage = 1x
		final price index = $5.01
		final pnl = 400.6%
	leverage = 1.2x
		final price index = $6.9
		final pnl = 590.0%
	>>>> ratio = 1.473 <<<<
SIGN = -1
	leverage = 1x
		final price index = $0.2
		final pnl = -80.2%
	leverage = 1.2x
		final price index = $0.14
		final pnl = -85.7%
	>>>> ratio = 1.068 <<<<

SIGN = 1
	leverage = 1x
		final price index = $5.01
		final pnl = 400.6%
	leverage = 0.8x
		final price index = $3.63
		final pnl = 263.0%
	>>>> ratio = 0.657 <<<<
SIGN = -1
	leverage = 1x
		final price index = $0.2
		final pnl = -80.2%
	leverage = 0.8x
		final price index = $0.27
		final pnl = -72.6%
	>>>> ratio = 0.905 <<<<

